In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from transformers import TextStreamer, TextIteratorStreamer
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
model_id = "google/gemma-2-2b-it"

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                        bnb_4bit_compute_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
    low_cpu_mem_usage=True,
)

chat = [
    { "role": "user", "content": "hello how are you?."},
]
question = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

question = tokenizer(question, return_tensors="pt").to(device)

streamer = TextStreamer(tokenizer, skip_prompt=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
streamer = TextStreamer(tokenizer, skip_prompt=True)

In [4]:
print(streamer)

In [6]:
_ = model.generate(**question, streamer=streamer,
                            pad_token_id=tokenizer.eos_token_id,
                            temperature=0.1,
                            max_length=2048,
                            do_sample=True,
                            top_p=0.5,
                            repetition_penalty=1.25)


I'm doing well, thank you! 😊  How can I help you today? 

Let me know if you have any questions or need anything. 😄 
<end_of_turn>


In [31]:
print(_)

tensor([[     2,      2,    106,   1645,    108,  17534,   1368,    708,    692,
          10481,    107,    108,    106,   2516,    108, 235285, 235303, 235262,
           3900,   1578, 235269,   7593,    692, 235341,  44416,    139,   2299,
            798,    590,   1707,    692,   3646, 235336, 235248,    109,   5331,
            682,   1230,   1013,    692,    791,   1089,   3920,    689,   1476,
           4341, 235265, 123781, 235248,    108,    107]], device='cuda:0')


In [30]:
decode_text = tokenizer.decode(_[0], skip_special_tokens=True)

In [29]:
print(decode_text)

user
hello how are you?.
model
I'm doing well, thank you! 😊  How can I help you today? 

Let me know if you have any questions or need anything. 😄 

